![image](https://raw.githubusercontent.com/IBM/watsonx-ai-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Use watsonx and IBM `granite-8b-code-instruct` to generate code based on instruction

#### Disclaimers

- Use only Projects and Spaces that are available in watsonx context.


## Notebook content

This notebook contains the steps and code to demonstrate support for code generation in watsonx.ai. It introduces commands for defining prompt and model testing.

Some familiarity with Python is helpful. This notebook uses Python 3.12.


## Learning goal

The goal of this notebook is to demonstrate how to generate code using `ibm/granite-8b-code-instruct` model based on instruction provided by the user.


## Contents

This notebook contains the following parts:

- [Setup](#setup)
- [Foundation Models in watsonx.ai](#models)
- [Generate code based on instruction](#generate)
- [Generated code testing](#testing)
- [Summary](#summary)

<a id="setup"></a>
## Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Contact with your Cloud Pak for Data administrator and ask them for your account credentials

### Install dependencies
**Note:** `ibm-watsonx-ai` documentation can be found <a href="https://ibm.github.io/watsonx-ai-python-sdk/index.html" target="_blank" rel="noopener no referrer">here</a>.

In [1]:
%pip install -U ibm-watsonx-ai | tail -n 1

#### Define credentials

Authenticate the watsonx.ai Runtime service on IBM Cloud Pak for Data. You need to provide the **admin's** `username` and the platform `url`.

In [2]:
username = "PASTE YOUR USERNAME HERE"
url = "PASTE THE PLATFORM URL HERE"

Use the **admin's** `api_key` to authenticate watsonx.ai Runtime services:

In [ ]:
import getpass
from ibm_watsonx_ai import Credentials

credentials = Credentials(
    username=username,
    api_key=getpass.getpass("Enter your watsonx.ai API key and hit enter: "),
    url=url,
    instance_id="openshift",
    version="5.2",
)

Alternatively you can use the **admin's** `password`:

In [3]:
import getpass
from ibm_watsonx_ai import Credentials

if "credentials" not in locals() or not credentials.api_key:
    credentials = Credentials(
        username=username,
        password=getpass.getpass("Enter your watsonx.ai password and hit enter: "),
        url=url,
        instance_id="openshift",
        version="5.2",
    )

### Working with projects

First of all, you need to create a project that will be used for your work. If you do not have project already created follow bellow steps.

- Open IBM Cloud Pak main page
- Click all projects
- Create an empty project
- Copy `project_id` from url and paste it below

**Action**: Assign project ID below

In [4]:
import os

try:
    project_id = os.environ["PROJECT_ID"]
except KeyError:
    project_id = input("Please enter your project_id (hit enter): ")

#### Create `APIClient` instance

In [5]:
from ibm_watsonx_ai import APIClient

client = APIClient(credentials, project_id)

<a id="models"></a>
## Foundation models in watsonx.ai

#### List available models

In [6]:
for model in client.foundation_models.TextModels:
    print(f"- {model}")

- ibm/granite-8b-code-instruct
- sdaia/allam-1-13b-instruct


You need to specify `model_id` that will be used for inferencing:

In [7]:
model_id = client.foundation_models.TextModels.GRANITE_8B_CODE_INSTRUCT

### Defining the model parameters

You might need to adjust model `parameters` for different models or tasks, to do so please refer to <a href="https://ibm.github.io/watsonx-ai-python-sdk/fm_model.html#metanames.GenTextParamsMetaNames" target="_blank" rel="noopener no referrer">documentation</a>.

In [8]:
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames

parameters = {
    GenTextParamsMetaNames.DECODING_METHOD: "greedy",
    GenTextParamsMetaNames.MAX_NEW_TOKENS: 100,
    GenTextParamsMetaNames.STOP_SEQUENCES: ["<end·of·code>"],
}

### Initialize the model
Initialize the model inference with previously set parameters.

In [9]:
from ibm_watsonx_ai.foundation_models import ModelInference

model = ModelInference(
    model_id=model_id, params=parameters, credentials=credentials, project_id=project_id
)

### Get model details

In [10]:
model.get_details()

{'model_id': 'ibm/granite-8b-code-instruct',
 'label': 'granite-8b-code-instruct',
 'provider': 'IBM',
 'source': 'IBM',
 'functions': [{'id': 'fine_tune_trainable'}, {'id': 'text_generation'}],
 'short_description': 'The Granite model series is a family of IBM-trained, dense decoder-only models, which are particularly well-suited for generative tasks.',
 'long_description': 'Granite models are designed to be used for a wide range of generative and non-generative tasks with appropriate prompt engineering. They employ a GPT-style decoder-only architecture, with additional innovations from IBM Research and the open community.',
 'terms_url': 'https://www.ibm.com/support/customer/csol/terms/?id=i126-6883',
 'input_tier': 'class_1',
 'output_tier': 'class_1',
 'number_params': '8b',
 'min_shot_size': 1,
 'task_ids': ['question_answering',
  'summarization',
  'classification',
  'generation',
  'extraction',
  'code-generation',
  'code-explanation',
  'code-fixing'],
 'tasks': [{'id': 'qu

<a id="generate"></a>
## Generate code based on instruction

Define instructions for the model with at-least one example.

In [11]:
instruction = """Using the directions below, generate Python code for the given task.
Input:
# Write a Python function that prints 'Hello World!' string 'n' times.

Output:
def print_n_times(n):
    for i in range(n):
        print("Hello World!")

<end of code>
"""

Prepare question for the model.

In [12]:
question = """Input:
# Write a Python function, which generates sequence of prime numbers.
# The function 'primes' will take the argument 'n', an int. It will return a list which contains all primes less than 'n'.
"""

### Generate the code using `ibm/granite-8b-code-instruct` model.

Inference the model to generate the code, according to provided instruction.

In [13]:
result = model.generate_text(f"{instruction} {question}")

Formatting the text to get the function itself

In [14]:
code_as_text = result.split("Output:")[1].split("<end of code>")[0]

<a id="testing"></a>
## Generated code testing

The resulting code looks as below.

In [15]:
print(code_as_text)


def primes(n):
    primes = []
    for num in range(2, n):
        is_prime = True
        for i in range(2, num):
            if num % i == 0:
                is_prime = False
                break
        if is_prime:
            primes.append(num)
    return primes




Use generated code to make it as function.

**Note**: Before executing this line, make sure the model's output visible above doesn't contain any malicious instructions.

In [16]:
exec(code_as_text)

Define the number 'n' for which the primes() function should process prime numbers.

In [17]:
n = 25

Test and run the generated function.

In [18]:
primes(n)

[2, 3, 5, 7, 11, 13, 17, 19, 23]

<a id="summary"></a>
## Summary and next steps

You successfully completed this notebook!
 
You learned how to generate code based on instruction with `ibm/granite-8b-code-instruct` in watsonx.ai. 
 
Check out our _<a href="https://ibm.github.io/watsonx-ai-python-sdk/samples.html" target="_blank" rel="noopener no referrer">Online Documentation</a>_ for more samples, tutorials, documentation, how-tos, and blog posts. 

### Authors

**Mateusz Szewczyk**, Software Engineer at watsonx.ai.

Copyright © 2023-2025 IBM. This notebook and its source code are released under the terms of the MIT License.